In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [20]:
def eiler(func, x0, xf, y0, h, args=()):
    count = int((xf - x0) / h) + 1
    y = np.zeros((count, y0.shape[0]))
    x, y[0] = x0, y0[:].copy()
    for i in range(1, count):
        right_parts = func(x, y[i-1], *args)
        for j in range(len(y0)):
            y[i][j] = y[i-1][j] + h * right_parts[j]
        x += h
    return y

In [15]:
def rk(func, x0, xf, y0, h, args=()):
    count = int((xf - x0) / h) + 1
    y = np.zeros((count, y0.shape[0]))
    x, y[0] = x0, y0[:].copy()
    for i in range(1, count):
        k1 = func(x, y[i-1], *args)
        k2 = func(x + h / 2, [y + k * h / 2 for y, k in zip(y[i-1], k1)], *args)
        k3 = func(x + h / 2, [y + k * h / 2 for y, k in zip(y[i-1], k2)], *args)
        k4 = func(x + h, [y + k * h for y, k in zip(y[i-1], k3)], *args)
        for j in range(len(y0)):
            y[i][j] = y[i-1][j] + h / 6 * (k1[j] + 2 * k2[j] + 2 * k3[j] + k4[j])
        x += h
    return y

In [13]:
def func(time: float, c: np.ndarray, 
         k: np.ndarray) -> np.ndarray:
    ca, cb, cc = c
    k1, k2, k3, k4 = k
    r1, r2, r3, r4 = [
        k1 * ca,
        k2 * cb ** 2,
        k3 * cb ** 2,
        k4 * cc,
    ]
    dca_dt = -r1 + r2
    dcb_dt = 2 * (r1 - r2 - r3 + r4)
    dcc_dt = r3 - r4

    return dca_dt, dcb_dt, dcc_dt

In [21]:
k, y0 = np.array([.2, .1, .1, .1]), np.array([1, .5, .2])
t0, tf = 0, 10
y_eiler = eiler(func, t0, tf, y0, 0.1, args=(k, ))
y_rk = rk(func, t0, tf, y0, 0.1, args=(k, ))

In [22]:
y_eiler

array([[1.        , 0.5       , 0.2       ],
       [0.9825    , 0.534     , 0.2005    ],
       [0.96570156, 0.56590376, 0.20134656],
       [0.94959   , 0.59574887, 0.20253557],
       [0.93414737, 0.62358651, 0.20405938],
       [0.91935302, 0.64947919, 0.20590738],
       [0.90518419, 0.67349853, 0.20806654],
       [0.89161651, 0.69572322, 0.21052188],
       [0.87862449, 0.71623708, 0.21325697],
       [0.86618195, 0.73512738, 0.21625435],
       [0.85426244, 0.75248326, 0.21949593],
       [0.8428395 , 0.76839443, 0.22296329],
       [0.83188701, 0.78295008, 0.22663795],
       [0.82137938, 0.79623788, 0.23050168],
       [0.81129174, 0.8083433 , 0.23453661],
       [0.80160009, 0.81934895, 0.23872543],
       [0.79228142, 0.82933415, 0.24305151],
       [0.78331374, 0.83837463, 0.24749894],
       [0.77467619, 0.84654228, 0.25205267],
       [0.766349  , 0.85390503, 0.25669849],
       [0.75831356, 0.86052681, 0.26142304],
       [0.75055235, 0.86646755, 0.26621387],
       [0.